In [1]:
from keras.models import load_model
from time import sleep, time
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from pytube import YouTube
import webbrowser
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
import random as r
import speech_recognition as sr
from transformers import pipeline


In [2]:
nltk.download('vader_lexicon')

data = pd.read_csv('Youtube Video Dataset-Copy1.csv')
data = data.dropna()
data['Category'] = data['Category'].str.lower()

vectorizer = TfidfVectorizer()
video_title_vectors = vectorizer.fit_transform(data['Title'])

model = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
model.fit(video_title_vectors)

def get_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_score = sid.polarity_scores(text)['compound']
    
    if sentiment_score >= 0.05:
        return 'happy'
    elif sentiment_score <= -0.05:
        return 'sadness'
    else:
        return 'neutral'

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ronak\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
C:\Users\ronak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_base.py:564: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


In [3]:
def recommend_videos(category):
    category = category.lower()
    user_category_vector = vectorizer.transform([category])
    distances, indices = model.kneighbors(user_category_vector)
    recommended_videos = [data.iloc[i]['Videourl'] for i in indices[0]]
    return recommended_videos

def play_first_link(links):
    linking=r.choice(links)
    video_url = "www.youtube.com" + linking
    yt = YouTube(video_url)
    video_title = yt.title
    thumbnail_url = yt.thumbnail_url
    webbrowser.open(video_url)
    print("Playing YouTube video:")
    print("Title:", video_title)
    print("Thumbnail URL:", thumbnail_url)

def play_music_based_on_sentiment(sentiment):
    mood_to_music = {
        "happy": ["/watch?v=QLuu0QwjFJo", "/watch?v=_ae2j9jZY_U", "/watch?v=xo2wlMGfgn0"],
        "joy": ["/watch?v=QLuu0QwjFJo", "/watch?v=_ae2j9jZY_U", "/watch?v=xo2wlMGfgn0"],
        "neutral": ["/watch?v=oVrioWUZgfU", "/watch?v=2a5rxG4YlpY", "/watch?v=CmHfWSxt0UQ"],
        "sadness": ["/watch?v=nVGNUIj31ks", "/watch?v=EVF_AuhJgLg"]
    }
    if sentiment in mood_to_music:
        video_id = r.choice(mood_to_music[sentiment])
        play_first_link([video_id])
    else:
        print("No music found for the given sentiment.")


In [4]:
def reply_message(text):
    classifier = pipeline(
        "text-classification",
        model="j-hartmann/emotion-english-distilroberta-base",
        top_k=None,
    )
    data = classifier(text)
    sentiment = data[0][0]["label"]
    return sentiment


def play(user_sentiment,user_choice):
    # user_choice = input("Enter 1 for music or 2 for videos: ")
    if user_choice == '1':
        play_music_based_on_sentiment(user_sentiment)
    elif user_choice == '2':
    
        print("Tell us what type of category you are interested in to watch the videos:")
        print("1. Manufacturing\n2. History\n3. Travel Blog\n4. Science & Technology\n5. Art & Music")
        cat = input("Enter the category number: ")
        
        
        if cat == '1':
            category = "Manufacturing"
        elif cat == '2':
            category = "History"
        elif cat == '3':
            category = "Travel Blog"
        elif cat == '4':
            category = "Science & Technology"
        elif cat == '5':
            category = "Art & Music"
        else:
            category = "Art & Music"  # Default to Art & Music if an invalid category is chosen
        
        recommended_videos = recommend_videos(category)
        play_first_link(recommended_videos)
    else:
        print("Invalid choice. Please enter either 1 or 2.")



In [5]:
def detect_emotion_and_play():
    face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    classifier = load_model('sentiment_video.h5')
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'happy', 'neutral', 'sadness', 'Surprise']
    cap = cv2.VideoCapture(0)
    start_time = time()
    duration = 10  # Set the duration to 30 seconds
    last_detected_emotion = None

    while time() - start_time < duration:
        _, frame = cap.read()
        labels = []
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

            if np.sum([roi_gray]) != 0:
                roi = roi_gray.astype('float') / 255.0
                roi = img_to_array(roi)
                roi = np.expand_dims(roi, axis=0)

                prediction = classifier.predict(roi)[0]
                last_detected_emotion = emotion_labels[prediction.argmax()]
                label_position = (x, y)
                cv2.putText(frame, last_detected_emotion, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, 'No Faces', (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Emotion Detector', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    print("Last detected emotion:", last_detected_emotion)
    user_recommendation = input("Enter 1 for music or 2 for videos: ")
    play(last_detected_emotion,user_recommendation)

In [6]:
user_choice = input("How do you want to check sentiment? Enter 1 for voice, 2 for text, 3 for video: ")
if user_choice == '1':
    # Voice input
    text = ""
    rs = sr.Recognizer()
    with sr.Microphone() as microphone:
        rs.adjust_for_ambient_noise(microphone)
        print("Listening to mic...")
        audio = rs.listen(microphone)
        print("Done listening")
    try:
        text = rs.recognize_google(audio)
        print("Audio content:", text)
        user_sentiment = reply_message(text)
        print("Detected sentiment:", user_sentiment)
        user_recommendation = input("Enter 1 for music or 2 for videos: ")
        play (user_sentiment,user_recommendation)
    except Exception as e:
        print("Error:", str(e))
elif user_choice == '2':
    # Text input
    user_input_text = input("Enter a sentence or text: ")
    user_sentiment = reply_message(user_input_text)
    print("Detected sentiment:", user_sentiment)
    user_recommendation = input("Enter 1 for music or 2 for videos: ")
    play(user_sentiment,user_recommendation)
    # play_first_link(recommended_videos)
elif user_choice == '3':
    # Video input
    detect_emotion_and_play()
else:
    print("Invalid choice. Please enter 1, 2, or 3.")



How do you want to check sentiment? Enter 1 for voice, 2 for text, 3 for video:  1


Listening to mic...
Done listening
Audio content: hello
Detected sentiment: neutral


Enter 1 for music or 2 for videos:  2


Tell us what type of category you are interested in to watch the videos:
1. Manufacturing
2. History
3. Travel Blog
4. Science & Technology
5. Art & Music


Enter the category number:  5


Playing YouTube video:
Title: Art and Music
Thumbnail URL: https://lh3.googleusercontent.com/pwtTw2GYpZ_3p1-mOC42gLytAZKPzun1HptZxhgHdhJRMTGpl4glm8H_9Wg_PnQCfGKnv_DVM4EWsGM=w544-h544-l90-rj
